In [6]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import read_csv
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import imblearn
from imblearn.over_sampling import SMOTE

In [7]:
df = read_csv('comp90051-2021-semester-2-project-1/train.csv', header = None)
data = df.values
strategy = {1:590, 2:590,3:590, 4:590, 5: 590, 
            6:590, 7:590,8:590, 9:590, 10: 590,
            11:200, 12:200,13:200, 14:200, 15: 200,
            16:200, 17:200,18:200, 19:200, 20: 200,
            21:200, 22:200,23:200, 24:200, 25: 200,
            26:200, 27:200,28:200, 29:200, 30: 200,
            31:200, 32:200,33:200, 34:200, 35: 200,
            36:200, 37:200,38:200, 39:200, 40: 200,
            41:200, 42:200,43:200, 44:200, 45: 200,
            46:200, 47:200,48:200, 49:200}

strategy2 = {1:200, 2:200,3:200, 4:200, 5: 200, 
            6:200, 7:200,8:200, 9:200, 10: 200,
            11:200, 12:200,13:200, 14:200, 15: 200,
            16:200, 17:200,18:200, 19:200, 20: 200,
            21:200, 22:200,23:200, 24:200, 25: 200,
            26:200, 27:200,28:200, 29:200, 30: 200,
            31:200, 32:200,33:200, 34:200, 35: 200,
            36:200, 37:200,38:200, 39:200, 40: 200,
            41:200, 42:200,43:200, 44:200, 45: 200,
            46:200, 47:200,48:200, 49:200}
# resample
oversample = SMOTE(sampling_strategy=strategy)
undersample = imblearn.under_sampling.RandomUnderSampler(sampling_strategy=strategy2)

# shuffle
x, y = df.iloc[:, 1:-1], df.iloc[:, [-1]]
x, y = shuffle(x, y)


X_over, y_over = oversample.fit_resample(x, y)
# print(y_over.value_counts())


x_final, y_final = undersample.fit_resample(X_over, y_over)
# print(y_final.value_counts())
x_scaled = preprocessing.scale(x_final)

# train test split - cross validation
# X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_over, test_size = 0.5)

# PCA
# from sklearn.decomposition import PCA
# estimator = PCA(n_components=100) 
# pca_X_train = estimator.fit_transform(X_train)
# pca_X_test = estimator.transform(X_test)


   





In [8]:
# this part of code is used for finding the best parameters C and gamma, but it takes too long for running so I just comment it out
# param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}

# rbf = svm.SVC(kernel='rbf', gamma=0.0001, C=100, decision_function_shape='ovr', probability=True)
# # rbf = rbf.fit(X_train, y_train.values.ravel())
# grid_search = GridSearchCV(rbf, param_grid, n_jobs = 8, refit=True, verbose=1)  
# grid_search.fit(X_train, y_train.values.ravel())
# best_parameters = grid_search.best_params_
# for para, val in list(best_parameters.items()):    
#     print(para, val)  
# model = SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], decision_function_shape='ovr', probability=True)
# print(best_parameters['C'], best_parameters['gamma'])
# model.fit(X_train, y_train.values.ravel())   

# rbf_pred = model.predict(X_test)
# accuracy_rbf = model.score(X_test, y_test)
# print('Accuracy Radial Basis Kernel:', accuracy_rbf)
# cm_rbf = confusion_matrix(y_test, rbf_pred)
# print(cm_rbf)

In [9]:

model = SVC(kernel='rbf', C=1000, gamma=0.001, decision_function_shape='ovr', probability=True)

model.fit(x_scaled, y_final.values.ravel())   

test_df = read_csv('comp90051-2021-semester-2-project-1/test.csv', header = None)
x_realtest = test_df.iloc[:, 1:]

# normalization
x_realtest_scaled = preprocessing.scale(x_realtest)

rbf_pred = model.predict(x_realtest_scaled)
df_pre = pd.DataFrame(rbf_pred, columns=['Category'])
df_pre.to_csv('predictions_9.csv')

# used for cross validation
# accuracy_rbf = model.score(X_test, y_test)
# print('Accuracy Radial Basis Kernel:', accuracy_rbf)




In [10]:
# this part of code is used for finding the best trainning set
from sklearn.model_selection import KFold
acc_score = []

kf = KFold(n_splits=10)

for train_index, test_index in kf.split(x_scaled, y_final):

    X_train, X_test = x_scaled[train_index], x_scaled[test_index]
    y_train, y_test = y_final.values.ravel()[train_index], y_final.values.ravel()[test_index]
    
    
    model.fit(X_train,y_train)

    acc_score.append(model.score(X_test, y_test))
    print(acc_score)
    print(train_index, test_index)

np.mean(acc_score)


KeyboardInterrupt: 